## Drude Model

Recall that the electric susceptability of our material is modeled by
$$
\chi_j(t)=e^{-\gamma_jt}\left[A_{1,j}e^{\beta_jt}+A_{2,j}e^{-\beta_jt}\right]
$$
where $\beta=\sqrt{\gamma^2-\omega_0^2}$. To obtain the Drude model we choose the natural frequency $\omega_0=0$ and $A_{1,j}=-A_{2,j}$ for all $j$, which reduces $\chi_j(t)$ to
$$
\chi_j(t)=A_{1,j}\left[1-e^{-2\gamma_jt}\right]
$$
How do we transform this into frequency space?
which results in
$$
\sigma(\omega)=\frac{\omega_0}{1+i\omega t}
$$

We start by importing the needed libraries and defining our simulation bounds and constants. Our simulation will begin at time index $-1$ps and end at time index $1.5$ps. The simulations spatial bounds will span from $-5$um to $10$um.

In [1]:
%matplotlib qt
# Imports
from rcfdtd_sim import Sim, Current, Mat, vis
import numpy as np
from scipy.fftpack import fft, fftfreq, fftshift
from matplotlib import pyplot as plt
from pathlib import Path
# Determine file save name
fsave = 'drude_model.npz'
# Constants
c0 = 1 # um/ps
di = 0.03 # 0.03 um
dn = di/c0 # (0.03 um) / (300 um/ps) = 0.0001 ps = 0.1 fs
epsilon0 = 1
mu0 = 1
# Define bounds
i0 = -5 # -5 um
i1 = 10 # 10 um
n0 = -300 # (0.1 fs) * (-300 um) / (0.03 um/step) = (0.1 fs) * (-10,000 steps) = -1,000 fs = -1 ps
n1 = 450 # (0.1 fs) * (450 um) / (0.03 um/step) = (0.1 fs) * (15,000 steps) = 1,500 fs = 1.5 ps
# Calculate dimensions
nlen, ilen = Sim.calc_dims(n0, n1, dn, i0, i1, di)
# Create a arrays that hold the value of the center of each cell
t = np.linspace(n0+dn/2, n1+dn/2, nlen, endpoint=False) * (10/3) # Multiply by 10/3 to get from um -> fs
z = np.linspace(i0+di/2, i1+di/2, ilen, endpoint=False)

In [2]:
print('nlen=%i, ilen=%i' % (nlen, ilen))

nlen=25000, ilen=502


## Setup Current

Specify the location of our current pulse in time and space

In [3]:
cp_loc_val = -2.5 # -2.5 um
cp_time_val = 0 # 0 fs

Determine the simulation indicies that correspond to these locations

In [4]:
# Find indicies
cp_loc_ind = np.argmin(np.abs(np.subtract(z, cp_loc_val)))
cp_time_ind = np.argmin(np.abs(np.subtract(t, cp_time_val)))
# Find start and end indicies in time
spread = int(500 / 0.1) # (500 fs) / (0.1 fs/step) = 5,000 steps
cp_time_s = cp_time_ind - spread
cp_time_e = cp_time_ind + spread

Create the current pulse

In [5]:
# Make pulse
cpulse = np.append(np.diff(np.diff(np.exp(-((t[cp_time_s:cp_time_e]-cp_time_val)**2)/(2e4)))), [0,0])
# Plot
plt.plot(t[cp_time_s:cp_time_e], cpulse)
plt.xlabel('time [fs]')
plt.ylabel('current [mA]')
plt.show()
# Create Current object
current = Current(nlen, ilen, cp_time_s, cp_loc_ind, cpulse)

## Setup Material

Specify the location of our material (which will be $5$um in length)

In [6]:
# Set material length
m_len = 5 # 5 um
# Set locations
m_s_val = 0
m_e_val = m_s_val + m_len

Calculate the starting and ending indicies of our material

In [7]:
m_s_ind = np.argmin(np.abs(np.subtract(z, m_s_val)))
m_e_ind = np.argmin(np.abs(np.subtract(z, m_e_val)))

Setup material behavior

In [8]:
# Set constants
a = np.complex64(1000)
gamma = np.complex64(1e0)
freq = np.complex64(0)
# Calculate beta
ang_gamma = np.complex64(gamma * 2 * np.pi)
omega = np.complex64(freq * 2 * np.pi)
beta = np.sqrt(np.add(np.square(ang_gamma), -np.square(omega)), dtype=np.complex64)
a1 = np.complex64(a/(2*beta))
a2 = np.complex64(-a/(2*beta))

In [9]:
print(gamma, beta, a1, a2)

(1+0j) (6.2831855+0j) (79.57747+0j) (-79.57747+0j)


Create our material behavior matrices

In [10]:
# Determine matrix length
mlen = m_e_ind - m_s_ind
# Create matrices
m = np.ones((1, mlen), dtype=np.complex64)
mgamma = m * ang_gamma
mbeta = m * beta
ma1 = m * a1
ma2 = m * a2

Create our material object

In [11]:
inf_perm = 4
material = Mat(dn, ilen, nlen, m_s_ind, inf_perm, ma1, ma2, mgamma, mbeta, storelocs=[1])

## Running the Simulation

Create and run our simulation (or load simulation if one already exists)

In [12]:
# Create Sim object
tqdmarg = {'desc': 'Executing simulation'}
s = Sim(i0, i1, di, n0, n1, dn, epsilon0, mu0, 'absorbing', current, material, nstore=int(nlen/50), storelocs=[5,ilen-6])
# Run simulation if simulation save doesn't exist
sim_file = Path(fsave)
if sim_file.is_file():
    # Load results
    dat = np.load(fsave)
    n = dat['n']
    ls = dat['ls']
    els = dat['els']
    erls = dat['erls']
    hls = dat['hls']
    hrls = dat['hrls']
    chi = dat['chi']
else:
    # Run simulation
    s.simulate(tqdmarg)
    # Export visualization
    #vis.timeseries(s, iunit='um', fname=fsave+'.mp4')
    # Export and save arrays
    n, ls, els, erls, hls, hrls = s.export_locs()
    ls_mat, chi = material.export_locs()
    n = n * (10/3) # 10/3 scale factor converts from um -> fs
    np.savez(fsave, n=n, ls=ls, els=els, erls=erls, hls=hls, hrls=hrls, chi=chi)

Plot fields in time

In [13]:
# Extract incident, transmitted, and reflected fields
inc = erls[:,1]
trans = els[:,1]
refl = els[:,0] - erls[:,0]

# Plot
plt.plot(n, inc, label='$E_i$')
plt.plot(n, trans, label='$E_t$')
plt.plot(n, refl, label='$E_r$')
plt.ylabel('Amplitude [?]')
plt.xlabel('time [fs]')
plt.legend()
plt.show()

/Users/jroth/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


Transform time-domain fields into frequency domain fields, extract transmission coefficient $\tilde T=A(\omega)+i\phi(\omega)$ into `spec_m` and `spec_a` arrays representing $A(\omega)$ and $\phi(\omega)$, respectively. To prevent a `divide by zero` error we remove the indicies at which the incident field $E_i(\omega)$ is zero.

In [14]:
# Calculate time difference
dn = np.diff(n)[0] # Calculate time step difference in fs

# Calculate Fourier transforms
freq = fftfreq(nlen, dn) * 1e3 # in THz (since [dn]=[fs], 1/[dn] = 1/[fs] = 10^15/[s] = 10^3*10^12/[s] = 10^4*[THz])
incf = fft(inc)
transf = fft(trans)

# Removeunwanted frequencies
freq = freq[1:int(nlen/2)]
incf = incf[1:int(nlen/2)]
transf = transf[1:int(nlen/2)]

# Remove zero indicies from all arrays
nonzero_ind = np.nonzero(incf)
freq = freq[nonzero_ind]
incf = incf[nonzero_ind]
transf = transf[nonzero_ind]

# Calculate spectrum in frequency
spec = np.divide(transf, incf)

# Remove zero indicies from all arrays
nonzero_ind = np.nonzero(spec)
freq = freq[nonzero_ind]
incf = incf[nonzero_ind]
transf = transf[nonzero_ind]
spec = spec[nonzero_ind]

# Extract phase and magnitude
spec_m = np.absolute(spec)
spec_a = np.abs(np.unwrap(np.angle(spec)))

# Plot
fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, dpi=100)
ax0.plot(freq, spec_m)
ax1.plot(freq, spec_a)
ax0.set_ylim(0, 1.5)
ax1.set_ylim(0, 2e2)
ax1.set_xlim(0, 1e2)
ax0.set_ylabel(r'$E_t(\omega)/E_i(\omega)$')
ax1.set_ylabel(r'$\phi(\nu)$ [rad]')
ax1.set_xlabel(r'Frequency [THz]')
plt.show()

We wish to calculate the index of refraction
$$\tilde{n}(\omega)=n(\omega)+i\kappa(\omega)$$
where $\kappa\ll n$. From Benjamin Ofori-Okai's 2016 PhD thesis Eq.(5.41) we note that conductivity is defined as
$$
\tilde{\sigma}(\omega)=\frac{2}{Z_0d}\left(\frac{1}{\tilde{t}(\omega)}-1\right)
$$
for a sample in vacuum where $Z_0$ is the impedance of free space, $d$ is the sample width, and $\tilde{t}(\omega)=\frac{E_t(\omega)}{E_i(\omega)}$. The index of refraction $\tilde{n}(\omega)$ is related to the conductivity $\tilde{\sigma}(\omega)$ via
$$
\tilde{n}(\omega)=\sqrt{\epsilon_\infty+\frac{i\tilde{\sigma}(\omega)}{\omega\epsilon_0}}
$$

This calculation is valid in the limit $\kappa\gg n$, or the _thin sample limit_. This simulation operates in this regime.

In [15]:
# Set constants
L = m_len * 1e-6
Z0 = 376.73 # Ohms (impedance of free space)
permittivity_free_space = 8.854187817e-12

# Calculate the angular frequency
ang_freq = 2 * np.pi * freq # THz * 2pi

# Calculate conductivity
conductivity = np.multiply(np.divide(2, Z0*L), np.subtract(np.divide(1, spec), 1))

# Calculate index of refraction
n_complex = np.sqrt(inf_perm + np.divide(np.multiply(1j, conductivity), np.multiply(ang_freq, permittivity_free_space)))

# Calculate the imaginary part of the index of refraction
n1 = np.real(n_complex)
kappa1 = np.imag(n_complex)

We finally plot these results

In [20]:
# Setup plot
fig = plt.figure(dpi=100)
fig.set_dpi(150)
ax0 = plt.gca()
ax0.set_xlabel(r'$\omega$ [$2\pi\times$THz]')
ax0.set_ylabel(r'$\mathcal{R}(\sigma)$')

# Plot conductivity
ax0.plot(ang_freq, np.real(conductivity), 'b-')
ax0.set_xlim(0, 2*np.pi*3)
#ax0.set_yscale('log')

(0, 18.84955592153876)

In [18]:
# Setup plot
fig = plt.figure(dpi=100)
fig.set_dpi(150)
ax0 = plt.gca()
ax0.set_title('Simulation')
ax0.set_xlabel(r'$\omega$ [$2\pi\times$THz]')
ax0.set_ylabel(r'$n$')
ax1 = ax0.twinx()
ax1.set_ylabel(r'$\kappa$')

# Plot n
n1_line, = ax0.plot(ang_freq, n1, 'b-')
ax0.set_xlim(0, 2*np.pi*3)
ax0.set_yscale('log')
#ax0.set_ylim(3.9, 4.1)

# Plot kappa
kappa1_line, = ax1.plot(ang_freq, kappa1, 'r--')
#ax1.set_yscale('log')
ax1.set_ylim(-0.5e7, 0.5e7)

# Post formatting and display
ax0.legend((n1_line, kappa1_line), ('$n$', '$\kappa$'), loc=1)
plt.tight_layout()
plt.show()

In [ ]:
# Setup figure
plt.close('all')
fig = plt.figure(figsize=(12, 8), dpi=60)

# Setup axes
ax_chi = plt.subplot2grid((5,3), (1, 0), 2, 1)
ax_freq = plt.subplot2grid((5,3), (1, 1), 2, 2)
ax_time = plt.subplot2grid((5,3), (0, 0), 1, 3)
ax_t = plt.subplot2grid((5,3), (3, 0), 1, 2, sharex = ax_freq)
ax_p = plt.subplot2grid((5,3), (4, 0), 1, 2, sharex = ax_t)
ax_n = plt.subplot2grid((5,3), (3, 2), 2, 1)
ax_k = ax_n.twinx()

# Time axis
ax_time.plot(n*1e-3, inc, label='$E_i(t)$')
ax_time.plot(n*1e-3, trans, label='$E_t(t)$')
ax_time.plot(n*1e-3, refl, label='$E_r(t)$')
ax_time.set_ylabel('amplitude [?]')
ax_time.set_xlabel('time [ps]')
ax_time.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0)

# Chi axis
ax_chi.plot(n,chi)
ax_chi.set_ylabel('current [?]')
ax_chi.set_xlabel('time [fs]')
ax_chi.set_xlim(-1e3,-990)
ax_chi.ticklabel_format(style='sci', scilimits=(0,0), axis='y')

# Frequency axis
ax_freq.plot(freq, np.abs(incf), label='$E_i(\omega)$')
ax_freq.plot(freq, np.abs(transf), label='$E_t(\omega)$')
ax_freq.set_ylabel('amplitude [?]')
ax_freq.set_xlabel('frequency [THz]')
ax_freq.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0)

# Coefficient plot
ax_t.plot(freq, spec_m)
ax_t.set_ylim(0,1.5)
ax_t.set_ylabel(r'$E_t(\omega)/E_i(\omega)$')

ax_p.plot(freq, spec_a)
ax_p.set_ylim(0,2e2)
ax_p.set_ylabel(r'$\phi(\nu)$ [rad]')
ax_p.set_xlabel(r'frequency [THz]')
ax_p.ticklabel_format(style='sci', scilimits=(0,0), axis='y')

# Index of refraction plot
n1_line, = ax_n.plot(ang_freq, n1, 'b-')
kappa1_line, = ax_k.plot(ang_freq, kappa1, 'r--')
ax_n.set_xlabel(r'$\omega$ [$2\pi\times$THz]')
ax_n.set_ylabel(r'$n$')
ax_k.set_ylabel(r'$\kappa$')
ax_n.set_ylim(1.9, 2.1)
ax_k.set_ylim(-0.2, 0.2)
ax_n.set_xlim(0, 2*np.pi*5e2)
ax_n.legend((n1_line, kappa1_line), ('$n$', '$\kappa$'), bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode='expand', borderaxespad=0)

# Final setup
ax_time.set_title(r'RC-FDTD Simulation: $\epsilon_\infty=16$, $\chi(\omega)=0$, L=$1.25$mm, THz pulse')
ax_p.set_xlim(0,5e2)
plt.tight_layout()

# Show figure
fig.savefig(fname='/Users/jroth/Downloads/export.pdf', format='pdf')
fig.show()